## **Importarea si citirea datelor**

In [ ]:
import pandas as pd
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_data.csv")

Vizualizarea datelor de antrenament

In [ ]:
df_train

Tratamentul valorilor lipsa

In [ ]:
df_train.isna().sum() # sau df_train.isnull().sum()

Verificam si pe df_train, cat si pe df_test

In [ ]:
df_train.loc[df_train['User_Behavior_Score'].isna(), 'User_Behavior_Score'] = df_train['User_Behavior_Score'].mean()
df_test.loc[df_test['User_Behavior_Score'].isna(), 'User_Behavior_Score'] = df_test['User_Behavior_Score'].mean()

Acest segment se ocupa de gestionarea valorilor lipsa din coloana User_Behavior_Score din ambele seturi de date (de antrenament si de test). In loc sa eliminam randurile care contin valori lipsa, inlocuim valorile lipsa cu media valorilor existente pentru date numerice sau cu cel mai frecvent element (moda) pentru datele categorice, folosind mean() pentru date numerice si mode()[0] pentru datele categorice. mode()[0] returneaza elementul care apare cel mai des in coloana.

**Subtask 1**

In [ ]:
df_test['Timestamp'] = pd.to_datetime(df_test['Timestamp'])
df_test['Period'] = df_test['Timestamp'].apply(lambda x: 'AM' if x.hour < 12 else 'PM')

Acest cod transforma coloana Timestamp intr-un obiect de tip data-timp folosind pd.to_datetime(). Apoi, pe baza orei din timestamp, se creeaza o coloana suplimentara Period care indica daca timpul este AM sau PM. Aceasta poate fi o caracteristica utila pentru a determina tipuri de comportamente in functie de perioada zilei.

In [ ]:
df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train['Period'] = df_train['Timestamp'].apply(lambda x: 'AM' if x.hour < 12 else 'PM')

In [ ]:
df_train.drop(columns=['Timestamp'], inplace=True)
df_test.drop(columns=['Timestamp'], inplace=True)

value_subtask1 = df_test['Period']
task1 = pd.DataFrame({
    'subtaskID': 1,
    'datapointID': df_test.ID,
    'answer':  value_subtask1
})

Dupa ce am obtinut informatia relevanta din Timestamp (adica perioada AM/PM), coloana Timestamp devine redundanta si este eliminata din ambele seturi de date.
Pentru subtask-ul 1, este creat un DataFrame care contine ID-ul fiecarui datapoint din setul de test (df_test.ID) si valoarea predictiei pentru fiecare datapoint (in acest caz, perioada AM/PM). Aceasta informatie este regasita in starter kit.

**Preprocesarea etichetelor (label encoding)**

Am verificat folosind df_train.info() daca avem coloane de tipul Object pentru a le preprocesa. Acest pas ne ajuta sa identificam coloanele care contin date de tip text sau categorii, astfel incat sa putem aplica tehnici de preprocesare, cum ar fi transformarea acestora in valori numerice.

In [ ]:
df_train.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['Period'] = le.fit_transform(df_train['Period'])
df_test['Period'] = le.transform(df_test['Period'])

In acest pas, coloana Period (care contine valori de tip string: 'AM' si 'PM') este transformata in valori numerice folosind LabelEncoder. Acest lucru este necesar deoarece majoritatea modelelor de invatare automata necesita ca datele de intrare sa fie numerice.

**Impartirea setului de date in seturi de antrenament si validare**

Setul de date de antrenament este impartit intr-un set de antrenament (80%) si un set de validare (20%) folosind train_test_split. Aceasta ajuta la evaluarea modelului pe datele care nu au fost folosite pentru antrenament, prevenind astfel overfitting-ul si asigurand o estimare mai corecta a performantelor modelului pe date noi.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train, validation = train_test_split(df_train, test_size=0.2, random_state=0)

y_train = train['Attack Type']
X_train = train.drop(columns=['Attack Type'])

y_validation = validation['Attack Type']
X_validation = validation.drop(columns=['Attack Type'])

**Subtask2**

**Antrenarea modelului + Evaluarea performanței modelului**

Campul tinta al acestui subtask este Attack Type: Predictia tipului de atac cibernetic. Scopul este sa antrenam un model de invatare automata pentru a prezice tipul de atac cibernetic (Attack Type) pe baza celorlalte caracteristici din setul de date. Obiectivul este de a clasifica corect fiecare inregistrare ca fiind unul dintre tipurile de atacuri cibernetice posibile, utilizand caracteristicile disponibile in setul de date.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

model = RandomForestClassifier(n_estimators=100)
model = model.fit(X_train, y_train)
y_pred = model.predict(X_validation)
print(f1_score(y_pred, y_validation, average='macro'))

Aici, se antreneaza un model de clasificare utilizand un Random Forest cu 100 de arbori de decizie (n_estimators=100). Random Forest este un model robust care combina mai multi arbori de decizie pentru a obtine predictii mai precise, reducand riscul de overfitting si imbunatatind generalizarea modelului.

Performanta modelului este evaluata folosind metrica F1-score (media armonica intre precizie si recall). F1-score se calculeaza pe setul de validare, ceea ce ajuta la masurarea performantei modelului pe datele care nu au fost folosite pentru antrenament, oferind astfel o evaluare echilibrata a capacitatii modelului de a identifica corect atat cazurile pozitive, cat si pe cele negative. F1-score este impusa de problema pentru a evalua echilibrat performanta modelului.

F1-score pe acest model: 0.87385138542436

**Predictia pentru setul de testare**

In [ ]:
y_pred_test = model.predict(df_test)

task2 = pd.DataFrame({
    'subtaskID': 2,
    'datapointID': df_test.ID,
    'answer': y_pred_test
})

Dupa ce modelul este antrenat si evaluat, se realizeaza predictiile pentru setul de testare (df_test). Rezultatele sunt stocate intr-un DataFrame similar cu cel creat pentru subtask-ul 1.

**Generarea fisierului de iesire**

In [ ]:
submission_df = pd.concat([task1, task2], ignore_index=True)
submission_df.to_csv("\submission.csv", index=False)

In final, cele doua subtasks (1 si 2) sunt combinate intr-un singur DataFrame si sunt salvate intr-un fisier CSV, care va fi utilizat pentru a trimite predictiile.

George (george_8074 pe discord)
 Daca ai intrebari sau ceva nu este clar, astept un mesaj!